In [1]:
%matplotlib inline

In [2]:
from functools import partial

import numpy as np
from qdev_wrappers.dataset.doNd import do0d, do1d, do2d


from qcodes.dataset.database import initialise_database
from qcodes.dataset.experiment_container import new_experiment
from qcodes.dataset.plotting import plot_by_id
import plottr
import qcodes.config
from qcodes.tests.instrument_mocks import DummyChannelInstrument

# Live plotting

This should perform live plotting automatically provided that you have plottr or qcplot installed and running. Plotter can be found at https://github.com/kouwenhovenlab/plottr and QCPlot in Teams. The live plotting needs to be configured as descried in in https://qcodes.github.io/Qcodes/examples/plotting/live_plotting.html



In [3]:
initialise_database()  # just in case no database file exists
new_experiment("doNd-tutorial", sample_name="no sample")

doNd-tutorial#no sample#10@C:\Users\jenielse/experiments.db
-----------------------------------------------------------

In [4]:
# preparatory mocking of physical setup
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])
channlinstrument = DummyChannelInstrument('arrayinst')

In [5]:
# and we'll make a 2D gaussian to sample from/measure
def gauss_model(x0: float, y0: float, sigma: float, noise: float=0.0005):
    """
    Returns a generator sampling a gaussian. The gaussian is
    normalised such that its maximal value is simply 1
    """
    while True:
        (x, y) = yield
        model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
        noise = np.random.randn()*noise
        yield model + noise

In [6]:
# and finally wire up the dmm v1 to "measure" the gaussian

gauss = gauss_model(0.1, 0.2, 0.25)
next(gauss)

def measure_gauss(dac):
    val = gauss.send((dac.ch1.get(), dac.ch2.get()))
    next(gauss)
    return val

dmm.v1.get = partial(measure_gauss, dac)
dmm.v2.get = np.random.rand

In [7]:
dataid, ax, cbax = do1d(dac.ch1, 0, 1, 100, 0.1, dmm.v1, dmm.v2)

Starting experimental run with id: 27
plot by id took 0.19638419151306152


In [8]:
dataid, ax, cbax = do2d(dac.ch1, 0, 1, 100, 0, 
                        dac.ch2, 0, 0.5, 100, 0, 
                        dmm.v1, dmm.v2,
                        do_plot=True)

Starting experimental run with id: 28
plot by id took 1.528921365737915


# Adding some actions

Below we will demonstarte that it's possible to execute actions both before and after each stage on the loops. We will leave these actions as simple prints so that one can see when they are triggered.

In [9]:
def before_action():
    print("called before the loop")

def after_action():
    print("called after the loop")
    
def at_each_action():
    print("called at each step")
    
def before_inner_action():
    print("called before inner loop")
    
def after_inner_action():
    print("called after inner loop")

In [10]:
dataid = do1d(dac.ch1, 0, 1, 10, 0.01, dmm.v1, dmm.v2, at_each_action,
              enter_actions=(before_action,), exit_actions=(after_action,))

called before the loop
Starting experimental run with id: 29
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called after the loop
plot by id took 0.1400294303894043


In [11]:
dataid = do2d(dac.ch1, 0, 1, 3, 0.01, 
              dac.ch2, 0, 0.5, 2, 0.01, 
              dmm.v1, dmm.v2, at_each_action,
              before_inner_actions=(before_inner_action,),
              after_inner_actions=(after_inner_action,),
              enter_actions=(before_action,), exit_actions=(after_action,))

called before the loop
Starting experimental run with id: 30
called before inner loop
called at each step
called at each step
called after inner loop
called before inner loop
called at each step
called at each step
called after inner loop
called before inner loop
called at each step
called at each step
called after inner loop
called after the loop
plot by id took 0.18652963638305664


# Array and MultiParameters

In [12]:
channlinstrument.A.dummy_array_parameter.get = lambda: np.random.rand(5)

In [13]:
dataid = do1d(dac.ch1, 0, 1, 10, 0.01, channlinstrument.A.dummy_array_parameter)

Starting experimental run with id: 31
plot by id took 0.15599322319030762


## do0D

Perhaps we just want to capture the arrayparameter without doing any sweep

dataid = do0d(channlinstrument.A.dummy_array_parameter)

Now lets try capturing 2 Array parameters.
First lets change the setpoints of one of them a bit to make them standout from each other

In [22]:
channlinstrument.B.dummy_array_parameter.setpoints = ((9.0, 10.0, 11.0, 12.0, 13.0),)

In [23]:
dataid = do0d(channlinstrument.A.dummy_array_parameter, channlinstrument.B.dummy_array_parameter)

Starting experimental run with id: 35
plot by id took 0.07499814033508301
